<a href="https://colab.research.google.com/github/Makorg123/Python-EDA/blob/main/HRAnalyticsUsingPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=fa43817bb81e13f330cdf0f464d30a4d25571137d36dbc4fe04b1300a0d274f6
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("classification").getOrCreate()

In [ ]:
hrdata = spark.read.csv("/content/drive/MyDrive/train_1.csv",inferSchema=True,header=True)

In [ ]:
hrdata.groupBy('education').count().show()

+----------------+-----+
|       education|count|
+----------------+-----+
|            null| 2409|
| Below Secondary|  805|
|Master's & above|14925|
|      Bachelor's|36669|
+----------------+-----+



In [ ]:
hrdata = hrdata.na.fill(value="Bachelor's",subset=['education'])

In [ ]:
hrdata.groupBy('previous_year_rating').count().show()

+--------------------+-----+
|previous_year_rating|count|
+--------------------+-----+
|                null| 4124|
|                   1| 6223|
|                   3|18618|
|                   5|11741|
|                   4| 9877|
|                   2| 4225|
+--------------------+-----+



In [ ]:
hrdata = hrdata.na.fill(value=3,subset=['previous_year_rating'])

In [ ]:
hrdata.groupBy('previous_year_rating').count().show()

+--------------------+-----+
|previous_year_rating|count|
+--------------------+-----+
|                   1| 6223|
|                   3|22742|
|                   5|11741|
|                   4| 9877|
|                   2| 4225|
+--------------------+-----+



In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [ ]:
indexer = [StringIndexer(inputCol = col, outputCol = col + "index").fit(hrdata)
for col in list(set(hrdata.columns)-set(['no_of_trainings','age',
                                         'length_of_service','avg_training_score','employee_id']))]

In [ ]:
pipeline = Pipeline(stages=indexer)

In [ ]:
hrdatadf = pipeline.fit(hrdata).transform(hrdata)

In [ ]:
hrdatadf.columns

['employee_id',
 'department',
 'region',
 'education',
 'gender',
 'recruitment_channel',
 'no_of_trainings',
 'age',
 'previous_year_rating',
 'length_of_service',
 'KPIs_met >80%',
 'awards_won?',
 'avg_training_score',
 'is_promoted',
 'KPIs_met >80%index',
 'educationindex',
 'is_promotedindex',
 'recruitment_channelindex',
 'previous_year_ratingindex',
 'genderindex',
 'employee_idindex',
 'regionindex',
 'awards_won?index',
 'departmentindex']

In [ ]:
columnstodrop = ['employee_id',
 'department',
 'region',
 'education',
 'gender',
 'recruitment_channel','previous_year_rating','KPIs_met >80%',
                 'awards_won?','is_promoted']

In [ ]:
hrdatadf = hrdatadf.drop(*columnstodrop)

In [ ]:
from pyspark.ml.feature import RFormula

In [ ]:
formula = RFormula(formula="is_promotedindex~.",featuresCol = 'features',labelCol = 'label')

In [ ]:
hrdatadf = formula.fit(hrdatadf).transform(hrdatadf)

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator

In [ ]:
logit = LogisticRegression()

In [ ]:
logitmodel = logit.fit(hrdatadf)

In [ ]:
logitmodel.summary.accuracy

0.9209057071960298

In [ ]:
logitpredict = logitmodel.transform(hrdatadf)

In [ ]:
accuracy = MulticlassClassificationEvaluator(metricName="accuracy")

In [ ]:
logitmodel.summary.areaUnderROC

0.8074738803330115

In [ ]:
auc = BinaryClassificationEvaluator()

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier(maxBins=35)

In [ ]:
treemodel = tree.fit(hrdatadf)

IllegalArgumentException: ignored